In [4]:
from abc import ABC, abstractmethod
from typing import Any, Callable, Optional, Union

import numpy as np
import numpy.typing as npt

In [5]:
class Parameter:
    def __init__(self, name: str, distribution: "Distribution"):
        """
        Records a name for a parameter and a distribution from which the parameter
        will be sampled.
        """
        # If a parameter is generated by a leaf distribution, error
        if isinstance(distribution, LeafDistribution):
            raise ValueError("Parameters must be generated by a non-leaf distribution")

        # Build a parameter with a name and a distribution
        self.name = name
        self.distribution = distribution

        # Note in the distribution that this parameter is generated by that distribution
        self.distribution.generated_parameters.append(self)

    def __str__(self):
        return f"{self.name} ~ {self.distribution}"

In [9]:
class Distribution:
    """Abstract class for distributions used in DMS Stan"""

    def __init__(
        self,
        numpy_dist: str,
        stan_to_np_names: dict[str, str],
        *,
        seed: int = 2,
        **parameters
    ):
        """Sets up random number generation and handles all parameters"""

        # Define variables with types that are created within this function
        self.rng: np.random.Generator
        self.numpy_dist: Callable[..., np.ndarray]
        self.parameters: dict[str, Parameter] = {}
        self.constants: dict[str, Any] = {}
        self.generated_parameters: list[Parameter] = []

        # Define the random number generator
        self.rng = np.random.default_rng(seed)

        # Store the numpy distribution
        self.numpy_dist = getattr(self.rng, numpy_dist)

        # Populate the parameters and constants
        for paramname, val in parameters.items():
            if isinstance(val, Parameter):
                self.parameters[paramname] = val
            else:
                self.constants[paramname] = val

        # All parameter names must be in the stan_to_np_names dictionary
        if missing_names := set(parameters.keys()) - set(stan_to_np_names.keys()):
            raise ValueError(
                f"Missing names in stan_to_np_names: {', '.join(missing_names)}"
            )

        # Store the stan names to numpy names dictionary
        self.stan_to_np_names = stan_to_np_names

    def sample(self, n: int) -> np.ndarray:
        """Sample from the distribution"""
        # Sample from the parameter distributions
        param_draws = {
            self.stan_to_np_names[name]: param.distribution.sample(n)
            for name, param in self.parameters.items()
        }

        # Add on constants
        param_draws.update(
            {
                self.stan_to_np_names[name]: val
                for name, val in self.constants.items()
            }
        )

        # Sample from this distribution using numpy
        return self.numpy_dist(**param_draws, size=n)

    def __str__(self):
        return f"{self.__class__.__name__}"

class Normal(Distribution):
    """Defines the normal distribution."""
    def __init__(self, *, mu: Union[Parameter, float], sigma: Union[Parameter, float], **kwargs):
        # Sigma must be positive
        if not isinstance(sigma, Parameter) and sigma <= 0:
            raise ValueError("`sigma` must be positive")

        super().__init__(
            numpy_dist="normal",
            stan_to_np_names={"mu": "loc", "sigma": "scale"},
            mu=mu,
            sigma=sigma,
            **kwargs
        )

class HalfNormal(Normal):
    """Defines the half-normal distribution."""
    def __init__(self, *, sigma: Union[Parameter, float], **kwargs):
        super().__init__(mu=0, sigma=sigma, **kwargs)

    # Overwrite the sample method to ensure that the sampled values are positive
    def sample(self, n: int) -> np.ndarray:
        return np.abs(super().sample(n))

class UnitNormal(Normal):
    """Defines the unit normal distribution."""
    def __init__(self, **kwargs):
        super().__init__(mu=0, sigma=1, **kwargs)

class Binomial(Distribution):
    """Defines the binomial distribution."""
    def __init__(self, *, theta: Union[Parameter, float], N: Union[Parameter, int], **kwargs):
        # Theta must be between 0 and 1
        if not isinstance(theta, Parameter) and not 0 <= theta <= 1:
            raise ValueError("`theta` must be between 0 and 1")

        super().__init__(
            numpy_dist="binomial",
            stan_to_np_names={"N": "n", "theta": "p"},
            N=N,
            theta=theta,
            **kwargs
        )

class Poisson(Distribution):
    """Defines the Poisson distribution."""
    def __init__(self, *, lambda_: Union[Parameter, float], **kwargs):

        # Lambda must be positive
        if not isinstance(lambda_, Parameter) and lambda_ <= 0:
            raise ValueError("`lambda_` must be positive")

        super().__init__(
            numpy_dist="poisson",
            stan_to_np_names={"lambda_": "lam"},
            lambda_=lambda_,
            **kwargs
        )

class Multinomial(Distribution):
    """Defines the multinomial distribution."""
    def __init__(
        self,
        *,
        theta: Union[Parameter, npt.ArrayLike],
        N: Optional[Union[Parameter, int]] = None,
        **kwargs
    ):
        # Sample the theta parameter if it is a distribution
        if isinstance(theta, Parameter):
            sampled = np.array(theta.distribution.sample(1))
            sampled = sampled.unsqueeze(0) if sampled.ndim == 1 else sampled

        # Otherwise make sure that it is a 1D array
        else:
            sampled = np.array(theta).unsqueeze(0)
            if sampled.ndim != 2:
                raise ValueError("Thetas must be passed as a 1D array")

        # Whether passed as a parameter or not, the thetas must sum to 1
        if not np.allclose(sampled.sum(axis = -1), 1):
            raise ValueError("All arrays of thetas must sum to 1")

        # Run the parent class's init
        super().__init__(
            numpy_dist="multinomial",
            stan_to_np_names={"N": "n", "theta": "pvals"},
            N=N,
            theta=theta,
            **kwargs
        )

    def sample(self, n: int) -> np.ndarray:
        # There must be a value for `N` in the parameters if we are sampling
        if (self.parameters.get("N") is None) and (self.constants.get("N") is None):
            raise ValueError(
                "Sampling from a multinomial distribution is only possible when "
                "'N' is provided'"
            )

        return super().sample(n)

class Beta(Distribution):
    """Defines the beta distribution."""
    def __init__(self, *, alpha: Union[Parameter, float], beta: Union[Parameter, float], **kwargs):
        super().__init__(
            numpy_dist="beta",
            stan_to_np_names={"alpha": "a", "beta": "b"},
            alpha=alpha,
            beta=beta,
            **kwargs
        )

class Dirichlet(Distribution):
    """Defines the Dirichlet distribution."""
    def __init__(self, *, alpha: Union[Parameter, npt.ArrayLike], **kwargs):
        super().__init__(
            numpy_dist="dirichlet",
            stan_to_np_names={"alpha": "alpha"},
            alpha=alpha,
            **kwargs
        )

In [8]:
dist.sample(10)

array([[ 8, 18, 18, 56],
       [11, 22, 25, 42],
       [ 5, 19, 33, 43],
       [11, 19, 31, 39],
       [10, 25, 31, 34],
       [ 8, 26, 28, 38],
       [11, 15, 32, 42],
       [14, 21, 32, 33],
       [11, 19, 22, 48],
       [11, 18, 28, 43]])